In [1]:
from pandas import Series, DataFrame, Panel
import numpy as np
import requests
import requests.auth
import pandas as pd
from newspaper import Article 
import praw
import json
with open('config.json') as f:
    data = json.load(f)
reddit_cred = data['Reddit']
watson_cred = data['Watson']
newspaper_cred = data['News']
sql_cred = data['SQL']




In [2]:
from sqlalchemy import create_engine
conn_string = 'mysql://{user}:{password}@{host}/{db}?charset=utf8mb4'.format(
    host = sql_cred["host"], 
    user = sql_cred["user"],
    password = sql_cred["password"], 
    db = 'MemeNews')
engine = create_engine(conn_string)

q = '''select * from every_comment'''

In [3]:
df = pd.read_sql(q, con = engine)


In [4]:
df['created'] = pd.to_datetime(df['created'], unit='s')
# df = df.set_index('created')


In [5]:
documents = list(df.body)

In [6]:

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

no_features = 1000

# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(documents)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()
tf_feature_names

['000',
 '10',
 '100',
 '11',
 '12',
 '15',
 '18',
 '20',
 '2017',
 '2018',
 '21',
 '22',
 '30',
 '4chan',
 '50',
 'ability',
 'able',
 'absolutely',
 'abuse',
 'accept',
 'access',
 'according',
 'account',
 'act',
 'action',
 'actions',
 'active',
 'actively',
 'actual',
 'actually',
 'administration',
 'admit',
 'age',
 'ago',
 'agree',
 'alcohol',
 'alien',
 'aliens',
 'allow',
 'allowed',
 'alt',
 'amendment',
 'america',
 'american',
 'americans',
 'animal',
 'animals',
 'answer',
 'anti',
 'anymore',
 'apparently',
 'apply',
 'area',
 'areas',
 'aren',
 'arent',
 'argue',
 'arguing',
 'argument',
 'arms',
 'article',
 'articles',
 'ask',
 'asked',
 'asking',
 'ass',
 'assault',
 'asshole',
 'assume',
 'assuming',
 'asylum',
 'attack',
 'attacks',
 'attempt',
 'attention',
 'authority',
 'automatic',
 'average',
 'avoid',
 'aware',
 'away',
 'babies',
 'baby',
 'bad',
 'ballots',
 'ban',
 'banned',
 'based',
 'basic',
 'basically',
 'bear',
 'behavior',
 'beliefs',
 'believe',
 '

In [ ]:
from sklearn.decomposition import NMF, LatentDirichletAllocation

no_topics = 20

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

# Run LDA
lda = LatentDirichletAllocation(n_components=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print ('''Topic+ '''+str(topic_idx)+' '.join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))
        print ()

no_top_words = 10
display_topics(nmf, tfidf_feature_names, no_top_words)
display_topics(lda, tf_feature_names, no_top_words)
display_topics(nmf, tfidf_feature_names, no_top_words)


In [ ]:
import pyLDAvis
import pyLDAvis.sklearn

pyLDAvis.sklearn.prepare(lda, df['body'], dtm_vectorizer, mds='tsne')
